<h2>Get the distances between each cardiac arrest and the nearest existing AED location<h2>

In [173]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
distances = np.load('./transformed_data/intervention_to_aed_distances_km.npy')
min_distances = np.min(distances, axis=1)

<h2>Get the distances between each cardiac arrest and the nearest potential AED location<h2>

In [174]:
intervention_coordinates = pd.read_csv("./transformed_data/intervention_coordinates.csv")
potential_aed_location = pd.read_csv("./transformed_data/potential_aed_locations.csv")
intervention_latlons = intervention_coordinates[['lat', 'lon']].values.tolist()
potential_aed_latlons = potential_aed_location[['lat', 'lon']].values.tolist()
intervention_latlons_radians = np.radians(np.array(intervention_latlons))
potential_aed_latlons_radians = np.radians(np.array(potential_aed_latlons))
intervention_to_potential_aed_distances_km = haversine_distances(intervention_latlons_radians, potential_aed_latlons_radians) * 6371000/1000
np.save("./transformed_data/intervention_to_potential_aed_distances_km.npy", intervention_to_potential_aed_distances_km)

In [175]:
new_distances = np.load('./transformed_data/intervention_to_potential_aed_distances_km.npy')
min_new_distances = np.min(new_distances, axis=1)

<h2> Find the cardiac arrests that are closer to potential AEDs than existing AEDs, and calculate the number of near cardiac arrests for each potential AED<h2>  

In [177]:
comparison_existing_potential = np.minimum(min_distances, min_new_distances)
potential_aed_index = np.argmin(new_distances, axis=1)
comparison_mask = (comparison_existing_potential == min_new_distances)
unique_potential_aed_index, intervention_counts= np.unique(potential_aed_index[comparison_mask], return_counts=True)
result_matrix = pd.DataFrame(np.vstack((unique_potential_aed_index, intervention_counts)).T, columns=['ID', 'Counts'])
result_matrix.set_index('ID', inplace=True)
potential_aed_location['arrest_counts'] = result_matrix.reindex(range(10000), fill_value=0)['Counts']
potential_aed_location['arrest_counts'] = potential_aed_location['arrest_counts'].astype(int)
potential_aed_location.to_csv('./transformed_data/potential_aed_locations&counts.csv', index=False)